#### 1. reads mapping && SNP calling && re-mapping

In [1]:
sh_dir = "/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/bam_wasp/"

#### 2. peak calling and remove m6Am

In [4]:
peak_dir = "/Charles/project/ASm6A/shFUS/mapping/human/peak/merged_peak_MSPC/"
rm_m6Am = "rm_m6Am"

#### 2. SNP filtering

In [1]:
# vcf_dir = "/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/SNP_calling/"
vcf_dir = "/Charles/project/ASm6A/shFUS/SNP_calling/06-CombinedVCF/"
os.chdir(vcf_dir)
############################################################################

#### select heterozygous SNP sites
# ! ls */*_gatk.vcf | parallel 'cat {} | java -jar /home/software/snpEff/SnpSift.jar filter "((countHet() > 0)" > {.}_het.vcf'
# ! ls */*_gatk_het.vcf | parallel "/home/cjr/software/gatk-4.1.2.0/gatk SelectVariants -R /home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/hg19.fa -V {} --select-type-to-include SNP --restrict-alleles-to BIALLELIC -O {.}_SNP.vcf"

### remove repeat && RNA editing
repeat = "/home/galaxy/project/alleleSpecific_analysis/data/RepeatMasker/RepeatMasker_hg19.record.bed"
editing = "/home/galaxy/project/alleleSpecific_analysis/data/RNA_editing/Human_AG_all_hg19_v2.bed"
#
result_dir = "filtered_vcf/rm_repeat_editing/"
os.system("mkdir -p %s" % result_dir)
vcf_list = glob.glob("*_het_SNP.vcf")
for vcf in vcf_list:
    result_file = os.path.join(result_dir, os.path.basename(vcf))
    os.system("bedtools intersect -a %s -b %s %s -v -wa -header > %s" % (vcf, repeat, editing, result_file))
    
#### overlapping with dbSNP
# dbsnp = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/dbsnp_138.hg19.bed"
dbsnp = "/home/Data/database/hg38/dbsnp_146.hg19.bed"
base_dir = "filtered_vcf/rm_repeat_editing/"
os.chdir(base_dir)
result_dir = "overlap_dbsnp/"
os.system("mkdir -p %s" % result_dir)
#
vcf_list = glob.glob("*_het_SNP.vcf") # ind1_gatk_het_SNP.vcf
for vcf in vcf_list:
    result_file = os.path.join(result_dir, os.path.basename(vcf))
    os.system("bedtools intersect -a %s -b %s -wa -u -header > %s" % (vcf, dbsnp, result_file))

In [5]:
### overlapping with m6A peak
vcf_dir = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/vcf_dir/filtered_vcf/"
# vcf_dir = "/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/SNP_calling/filtered_vcf/"
# vcf_dir = "/Charles/project/ASm6A/shFUS/SNP_calling/06-CombinedVCF/filtered_vcf/"
os.chdir(vcf_dir)
vcf_list = glob.glob("*.vcf")
result_dir = "overlap_peak/"
os.system("mkdir -p %s" % result_dir)
##
# peak_dir = "/Charles/project/ASm6A/shFUS/mapping/human/peak/merged_peak_MSPC/"
# peak_list = glob.glob("%s/*/ConsensusPeaks.bed"%peak_dir) # ind1_gatk_het_SNP.vcf
# peak_dir = "/Charles/project/ASm6A/shFUS/mapping/human/peak/merged_peak_MSPC/normalized_macs/"
peak_dir = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/peak/merged_peak_MSPC/normalized_macs/"
peak_list = glob.glob("%s/*.bed" % peak_dir)
#
# vcf_list = glob.glob("*_het_SNP.vcf") # ind1_gatk_het_SNP.vcf
# for vcf in vcf_list:
#     prefix = vcf.split("_het_SNP.vcf")[0]
#     peak = "%s/%s/ConsensusPeaks.bed" % (peak_dir, prefix)
#     result_file = os.path.join(result_dir, "%s_gatk.vcf"%prefix)
#     os.system("bedtools intersect -a %s -b %s -wa -u -header > %s" % (vcf, peak, result_file))
sample_dict = {"wt": "Nc1", "wt2": "NC", "kd": "Fus2", "kd2": "Fus"}
for peak in peak_list:
#     prefix = peak.split("/")[-2]
    prefix = os.path.basename(peak).split(".bed")[0]
    print(prefix)
#     vcf = "%s/inp_het_SNP.vcf" % (vcf_dir)
    vcf = [x for x in vcf_list if x.split("-")[0] == sample_dict[prefix]][0]
    result_file = os.path.join(result_dir, vcf)
    os.system("bedtools intersect -a %s -b %s -wa -u -header > %s" % (vcf, peak, result_file))

wt2
kd
kd2
wt


#### 3. Allele reads count

In [2]:
## Modify the address of the VCF file
# count_dir = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/readCount.sh"
# vcf_dir = "/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/SNP_calling/filtered_vcf/overlap_peak/"
vcf_dir = "/Charles/project/ASm6A/shFUS/SNP_calling/06-CombinedVCF/filtered_vcf/overlap_peak/"

#### 4. post filters and Hypothesis test

In [21]:
from scipy import stats
from multiprocessing import Pool
from statsmodels.stats.multitest import fdrcorrection

# os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/")
# os.chdir("/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/count/")
# os.chdir("/Charles/project/ASm6A/shFUS/SNP_calling/08-Count/")
os.chdir("/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/count/")
# os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/")
# sample_list = ["wt", "kd", "wt2", "kd2"]
# file_list = [["Nc1-ip_L4_711D01.readcounts.txt", "Nc1-inp_L4_709D01.readcounts.txt"],
#              ["Fus2-ip_L4_712D01.readcounts.txt", "Fus2-inp_L4_710D01.readcounts.txt"],
#              ["NC-IP_L1_706D07.readcounts.txt", "NC-inp_L1_704D07.readcounts.txt"],
#              ["Fus-IP_L1_707D07.readcounts.txt", "Fus-inp_L1_705D07.readcounts.txt"]]
# number_list = [[85184165, 78164647],
#                [84298167, 68023740],
#                [90201519, 46888237],
#                [117018661, 59254724]]
# sample_list = ["wt1", "wt2", "kd1-1", "kd1-2", "kd2-1", "kd2-2"]
# file_list = [["IP1_L3_701D08.readcounts.txt", "inp1_L3_707D08.readcounts.txt"],
#              ["IP2_L3_702D08.readcounts.txt", "inp2_L3_708D08.readcounts.txt"],
#              ["IP3_L3_703D08.readcounts.txt", "inp3_L3_709D08.readcounts.txt"],
#              ["IP4_L3_704D08.readcounts.txt", "inp4_L3_710D08.readcounts.txt"],
#              ["IP5_L3_705D08.readcounts.txt", "inp5_L3_711D08.readcounts.txt"],
#              ["IP6_L3_706D08.readcounts.txt", "inp6_L3_712D08.readcounts.txt"]]
# number_list = [[118311220, 23452971],
#               [120152621, 27208576],
#               [128702240, 28641536],
#               [96370568, 31142454],
#               [85093260, 24248087],
#               [97331100, 24074659]]
# sample_list = ["wt1", "wt2", "kd1-1", "kd1-2", "kd2-1", "kd2-2"]
# file_list = [["wt1_m6A.readcounts.txt", "wt1_input.readcounts.txt"],
#              ["wt2_m6A.readcounts.txt", "wt2_input.readcounts.txt"],
#              ["kd1-1_m6A.readcounts.txt", "kd1-1_input.readcounts.txt"],
#              ["kd1-2_m6A.readcounts.txt", "kd1-2_input.readcounts.txt"],
#              ["kd2-1_m6A.readcounts.txt", "kd2-1_input.readcounts.txt"],
#              ["kd2-2_m6A.readcounts.txt", "kd2-2_input.readcounts.txt"]]
# number_list = [[235340284, 98544569],
#               [234933232, 114472977],
#               [266382690, 108559562],
#               [174853943, 148216516],
#               [167459037, 116943118],
#               [193237130, 112129516]]
sample_list = ["nc", "nc2", "T1", "T2", "T3"]
file_list = [["nc.readcounts.txt", "Input-nc.readcounts.txt"],
             ["nc2.readcounts.txt","Input-nc2.readcounts.txt"],
             ["T1.readcounts.txt", "Input-t1.readcounts.txt"],
             ["T2.readcounts.txt", "Input-t2.readcounts.txt"],
             ["T3.readcounts.txt", "Input-t3.readcounts.txt"]]
#### WASP bam count
number_list = [[130757272, 111377376],
               [146388392, 107149644],
               [147709422, 126380464],
               [111946211, 43773006],
               [81784485, 109386167]]
# sample_list = ["nc", "T1", "T2", "T3"]
# file_list = [["nc_ip.readcounts.txt", "nc_input.readcounts.txt"],
#              ["T1_ip.readcounts.txt", "T1_input.readcounts.txt"],
#              ["T2_ip.readcounts.txt", "T2_input.readcounts.txt"],
#              ["T3_ip.readcounts.txt", "T3_input.readcounts.txt"]]
# #### WASP bam count
# number_list = [[65373607, 55697999],
#               [73850336, 63203740],
#               [55975782, 21890844],
#               [40891082, 54694449]]
##########
def preprocess_df(ip_file, input_file, totalNum_list):
    df_ip, df_input = pd.read_table(ip_file), pd.read_table(input_file)
    df_results = []
    treat_list, df_list = ['ip', 'input'], [df_ip, df_input]
    for i in range(len(df_list)):
        df, treat, total_num = df_list[i], treat_list[i], totalNum_list[i]
        df = df[[
            'contig', 'position', 'refAllele', 'altAllele', 'refCount',
            'altCount']]
        df['#contig'] = df['contig']
        del df['contig']
        ###########################################################
        if i == 0:  # ip sample: only restrict total number
            df = df[(df['refCount'] + df['altCount']) >= 25] #####
        else:
            df = df[(df['refCount'] >= 5) & (df['altCount'] >= 5)]
            df = df[(df['refCount'] + df['altCount']) >= 25] #####
        ###########################################################
        df['refCount_%s' % treat] = df['refCount']
        df['altCount_%s' % treat] = df['altCount']
        df['refRPKM_%s' % treat] = (df['refCount'] / total_num) * 1000000000
        df['altRPKM_%s' % treat] = (df['altCount'] / total_num) * 1000000000
        df_results.append(df)
    return df_results[0], df_results[1]

############ run
result_dir = "Hypothesis/Fisher/" ####
os.system("mkdir -p %s" % result_dir)
for i in range(len(sample_list)):
    result_file = os.path.join(result_dir, "%s.txt" % sample_list[i])
    df_ip, df_input = preprocess_df(file_list[i][0], file_list[i][1], number_list[i])
    print(len(df_ip), len(df_input))
    df = df_ip.merge(df_input,
                     on=['#contig', 'position', 'refAllele', 'altAllele'],
                     how='left').dropna(how="any")
    print(len(df))
    df['refRPKM_ratio'] = df['refRPKM_ip'] / df['refRPKM_input']
    df['altRPKM_ratio'] = df['altRPKM_ip'] / df['altRPKM_input']
    ## ref/(ref+alt)
    df['allelicRatio'] = df['refRPKM_ratio'] / (df['altRPKM_ratio'] +
                                                df['refRPKM_ratio'])
    ## Fisher's exact test
    pvalue_list, odds_list = [], []
    for i, j in df.iterrows():
        a, b = round(j['refRPKM_ip']) + 0.5, round(j['altRPKM_ip']) + 0.5
        c, d = round(j['refRPKM_input']) + 0.5, round(j['altRPKM_input']) + 0.5
        oddsratio, pvalue = stats.fisher_exact([[a, b], [c, d]])
        pvalue_list.append(pvalue)
        odds_list.append(oddsratio)
    ## FDR correction
    qvalue_list = list(fdrcorrection(pvalue_list)[1])
    df['pvalue'], df['oddsratio'], df[
        'qvalue'] = pvalue_list, odds_list, qvalue_list
    df_sub = df[[
        '#contig', 'position', 'refAllele', 'altAllele', 'refRPKM_ratio',
        'altRPKM_ratio', 'allelicRatio', 'pvalue', 'oddsratio', 'qvalue',
        'refCount_ip', 'altCount_ip', 'refCount_input', 'altCount_input'
    ]]
    df_sub.to_csv(result_file, sep="\t", index=False)

/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2201 1501
1499
2200 1489
1488
1735 1069
1068
1201 448
448
1710 1049
1047


#### 5. pick out final results

In [22]:
os.chdir(
    # "/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/count/Hypothesis/Fisher/"
#     "/Charles/project/ASm6A/shFUS/SNP_calling/08-Count/Hypothesis/Fisher/"
#     "/Charles/project/ASm6A/shFUS/SNP_calling/08-Count/Hypothesis/Fisher_2/" #####
#     "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/Fisher/"
    "/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/count/Hypothesis/Fisher/"
)
# result_dir = "filter/"
# os.system("mkdir -p %s" % result_dir)
txt_list = glob.glob("*.txt")
result_dir = "sig/"
os.system("mkdir -p %s" % result_dir)


########
def select_highFC(df, res):
    #     bed_list = glob.glob("%s/*.bed" % in_dir)
    #     for asm6a in bed_list:
    #     df = pd.read_table(in_file, header=None)
    #     df.columns = ['contig', 'start', 'position', 'term', 'refRPKM_ratio', 'altRPKM_ratio']
    # df['mark'] = df['term'].str.split(";").str[2]
    df['mark'] = np.where((df['refRPKM_ratio'] > df['altRPKM_ratio']), "ref",
                          "alt")
    ##### unsig
    df_u1 = df[df['qvalue'] >= 0.05]
    df_u2 = df[(df['qvalue'] < 0.05)
               & ((df['mark'] == "ref") & (df['refRPKM_ratio'] < 3))] ######
    df_u3 = df[(df['qvalue'] < 0.05)
               & ((df['mark'] == "alt") & (df['altRPKM_ratio'] < 3))] #######
    df_unsig = pd.concat([df_u1, df_u2, df_u3])
    df_unsig['mark'] = "unsig"
    ##### sig
    df_sig = df[df['qvalue'] < 0.05]
    df_sig = df_sig[((df_sig['mark'] == "ref") &
                     (df_sig['refRPKM_ratio'] >= 3)) | ############
                    ((df_sig['mark'] == "alt") &
                     (df_sig['altRPKM_ratio'] >= 3))]  ############
    ##### merge and write to file
    df = pd.concat([df_unsig, df_sig])
    df.to_csv(res, sep="\t", header=False, index=False)


for txt in txt_list:
    df = pd.read_table(txt)
    result_file = os.path.join(result_dir, txt)
    select_highFC(df, result_file)

In [23]:
### txt to bed
# os.chdir("/Charles/project/ASm6A/shFUS/SNP_calling/08-Count/Hypothesis/Fisher/sig/")
# os.chdir("/Charles/project/ASm6A/shFUS/SNP_calling/08-Count/Hypothesis/Fisher_2/sig/")
# os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/Fisher/sig/")
# os.chdir("/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/count/Hypothesis/Fisher/sig/")
# txt_list = glob.glob("*.txt")
# for txt in txt_list:
#     df = pd.read_table(txt, header=None)
#     df = df[df.iloc[:,-1] != "unsig"] ####
#     df['mark'] = df.iloc[:,-1]
#     df['chr'] = df.iloc[:,0]
#     df['s'] = df.iloc[:,1] - 1
#     df['e'] = df.iloc[:,1]
#     df[['chr','s','e','mark']].to_csv(txt.replace(".txt",".bed"), sep="\t", header=False, index=False)